In [108]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import time
from tqdm import tqdm
import json

In [97]:
def clean_text(text):
    # 不可視文字（例：U+2000–U+200F, U+FEFFなど）を削除
    return re.sub(r'[\u2000-\u200F\u3000-\u300F\uFEFF]', '', text)

def fetch_tax_answer(url):
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, "html.parser")

    # 本文取得（構造により変更の可能性あり）
    main_content = soup.select_one("div#bodyArea")
    if not main_content:
        main_content = soup.find("body")

    text_parts = []
    images = []

    target_tag_flag = False
    for element in main_content.descendants:
        if element.name in ["h2"] and element.text.strip()=="概要":
            target_tag_flag = True
            
        if not target_tag_flag:
            continue

        if element.name == "img":
            img_src = element.get("src")
            img_url = urljoin(url, img_src)
            images.append({
                "image_url": img_url,
                "context": element.find_parent().get_text(strip=True)  # 親要素の文脈を取得
            })
        elif element.name in ["p", "li", "div", "span"] and element.text:
            text = element.get_text(strip=True)
            if text:
                text = clean_text(text)
                if len(text) >= 10:
                    text_parts.append(text)
        if element.name in ["h2", "h3"] and element.text.strip()=="根拠法令等":
            break

    return {
        "text": "\n".join(text_parts),
        "images": images
    }

In [98]:
# 1. URLからHTMLを取得
url = "https://www.nta.go.jp/taxes/shiraberu/taxanswer/code/index.htm"
response = requests.get(url)
response.encoding = response.apparent_encoding  # 正しく文字コードを推定

In [99]:
# 2. BeautifulSoupでHTML解析
soup = BeautifulSoup(response.text, "html.parser")

In [100]:
# 3. タックスアンサーコードの一覧を抽出
answer_codes = []
for a_tag in soup.select("a[href^='/taxes/shiraberu/taxanswer/']"):
    href = a_tag.get("href")
    text = a_tag.get_text(strip=True)
    # コードと説明文に分ける（例: 1200 所得税の仕組み）
    if text and text[:4].isdigit():
        code = text[:4]
        description = text[4:].strip()
        answer_codes.append({
            "code": code,
            "description": description,
            "url": f"https://www.nta.go.jp{href}"
        })

answer_codes[:5]

[{'code': '1000',
  'description': '所得税のしくみ',
  'url': 'https://www.nta.go.jp/taxes/shiraberu/taxanswer/shotoku/1000.htm'},
 {'code': '1800',
  'description': 'パート収入はいくらまで所得税がかからないか',
  'url': 'https://www.nta.go.jp/taxes/shiraberu/taxanswer/shotoku/1800.htm'},
 {'code': '1810',
  'description': '家内労働者等の必要経費の特例',
  'url': 'https://www.nta.go.jp/taxes/shiraberu/taxanswer/shotoku/1810.htm'},
 {'code': '1190',
  'description': '配偶者の所得がいくらまでなら配偶者控除が受けられるか',
  'url': 'https://www.nta.go.jp/taxes/shiraberu/taxanswer/shotoku/1190.htm'},
 {'code': '1191',
  'description': '配偶者控除',
  'url': 'https://www.nta.go.jp/taxes/shiraberu/taxanswer/shotoku/1191.htm'}]

In [102]:
# 出力例
for entry in tqdm(answer_codes):  # 最初の10件のみ表示
    # print(f"{entry['code']} | {entry['description']} | {entry['url']}")
    result = fetch_tax_answer(entry['url'])
    entry["content"] = result

    time.sleep(2)

100%|██████████| 870/870 [28:52<00:00,  1.99s/it]


In [104]:
answer_codes[0]

{'code': '1000',
 'description': '所得税のしくみ',
 'url': 'https://www.nta.go.jp/taxes/shiraberu/taxanswer/shotoku/1000.htm',
 'content': {'text': '所得税は個人の所得に対してかかる税金であり1年間のすべての所得金額から所得控除額を差し引いた残りの金額（課税所得金額）に税率を適用して税額を計算します\n＜所得税および復興特別所得税の申告納税額の計算の流れ＞\n所得はその性質によって次の10種類に分かれそれぞれの所得について収入や必要経費の範囲あるいは所得の計算方法などが定められています\nなお所得税は納税義務者に帰属するすべての所得に対して課税することを原則としていますが所得の中には社会政策その他の見地から所得税を課さないもの（非課税所得）があります詳しくはコード2011課税される所得と非課税所得を参照してください\n課税所得金額はその方のすべての所得金額から所得控除額を差し引いて算出します所得控除とは控除の対象となる扶養親族が何人いるかなどの個人的な事情を加味して税負担を調整するもので次の種類があります\n4小規模企業共済等掛金控除\n所得税額は課税所得金額に所得税の税率を適用して計算しますなお所得税の税率は所得が多くなるに従って段階的に高くなり納税者がその支払能力に応じて公平に税を負担するしくみ（超過累進税率）となっています\n（注）土地建物等や株式等の譲渡所得など他の所得と区分して税額を計算する所得もあります詳しくはコード2240申告分離課税制度を参照してください\n所得税額から税額控除などの所得税額から差し引かれる金額を控除し基準所得税額を算出します主な税額控除は次のとおりです\n2政党等寄附金特別控除\n3認定NPO法人等寄附金特別控除\n4公益社団法人等寄附金特別控除\n5(特定増改築等)住宅借入金等特別控除\n平成25年から令和19年までの各年分については復興特別所得税を所得税と併せて申告・納付します復興特別所得税額は基準所得税額に2.1パーセントの税率を掛けて計算しますまた平成25年１月１日から令和19年12月31日までの間に生ずる所得については源泉所得税の徴収の際に復興特別所得税が併せて徴収されます\n基準所得税額と

In [105]:
char_length = 0
for entry in answer_codes:
    char_length += len(entry["content"]["text"])

In [107]:
char_length / 2

607119.5

In [109]:
def save_jsonl(data_list, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for item in data_list:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [110]:
save_jsonl(answer_codes, "../data/tax_answers.jsonl")